In [1]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import math
from itertools import combinations
import numpy as np
import time

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
class Instance(object):
    width = 0 #stands for the width of the plate
    n = 0 #stands for the number of the blocks in the instance
    dimensions = [] #stands for the dimensions of the blocks in the instance
    def __init__(self, width, n, dimensions):
        self.width = width
        self.n = n
        self.dimensions = dimensions

In [3]:
def read_file(file_name):
    dimensions = []
    with open(file_name) as f:
        width = int(f.readline())                 # Width of the plate
        n = int(f.readline())                     # Number of blocks
        while True:
            #.strip() has been used to solved instance 37 & 39 problem
            line=f.readline().strip()
            if not line: 
                break
            dimensions.append(line.split(" "))    # Dimensions of each plate
    for dim in dimensions:
        dim[0] = int(dim[0])
        dim[1] = int(dim[1])
    instance = Instance(width, n, dimensions)
    return instance

In [4]:
ins=1
while ins<41:

    instance_file = "instances\ins-{}.txt".format(ins)
    instance = read_file(instance_file)

    width = instance.width
    n = instance.n
    x_dims_init=[]
    y_dims_init=[]
        
    for x_dim,y_dim in instance.dimensions:
        x_dims_init.append(x_dim)
        y_dims_init.append(y_dim)


    min_height = sum([x_dims_init[i] * y_dims_init[i] for i in range(len(x_dims_init))])//width 
    
    max_height = math.ceil(sum(y_dims_init)/(instance.width//max(x_dims_init)))
    
    
    dims_upper_bound = max(max(x_dims_init), max(y_dims_init))
    
    M_x = width
    M_y = max_height
    
    biggest = np.argmax([x_dims_init[i] * y_dims_init[i] for i in range(len(x_dims_init))])
    
    #################
    #Model Definition
    #################
    
    model = pyo.ConcreteModel()
    
    #####################
    #Variables Definition
    #####################
    
    model.height = pyo.Var(within=Integers, bounds=(min_height,max_height))
        
    model.x = pyo.Var(list(range(len(x_dims_init))), domain=Integers, bounds=(0,width - min(x_dims_init)))  
    model.y = pyo.Var(list(range(len(y_dims_init))), domain=Integers, bounds=(0,max_height - min(y_dims_init)))

    
    model.x_dims = pyo.Var(list(range(len(x_dims_init))),within=Integers, bounds=(0,dims_upper_bound))
    
    model.y_dims = pyo.Var(list(range(len(y_dims_init))),within=Integers, bounds=(0,dims_upper_bound))
    
    model.rots = pyo.Var(list(range(len(x_dims_init))),within=pyo.Binary)
    
    model.ors = pyo.Var([(i,j,k) for (i,j) in combinations(model.x,2) for k in range(4)],within=pyo.Binary)
    
    #######################
    #Constraints Definition
    #######################
    
    model.x_rots = ConstraintList()    
    for i in list(range(len(model.x_dims))):
        x_rots_add=(model.x_dims[i] ==((1-model.rots[i]) * x_dims_init[i] + model.rots[i] * y_dims_init[i]))
        model.x_rots.add(expr=x_rots_add)
    
    model.y_rots = ConstraintList()    
    for i in list(range(len(model.x_dims))):
        y_rots_add=(model.y_dims[i] == ((1-model.rots[i]) * y_dims_init[i] + model.rots[i] * x_dims_init[i]))
        model.y_rots.add(expr=y_rots_add)
        
    model.x_constraints = ConstraintList()        
    for i in list(range(len(model.x_dims))):
        x_constraints_add=((model.x[i] + model.x_dims[i]) <= width)
        model.x_constraints.add(x_constraints_add)
        
    model.y_constraints = ConstraintList() 
    for i in list(range(len(model.y_dims))):
        model.y_constraints_add=((model.y[i] + model.y_dims[i]) <= model.height)
        model.y_constraints.add(model.y_constraints_add)

    model.at_least_one=ConstraintList()
    for i,j in combinations(model.x,2):
        at_least_one_expr=(model.ors[i,j,0]+model.ors[i,j,1]+model.ors[i,j,2]+model.ors[i,j,3])
        model.at_least_one.add(expr=at_least_one_expr>=1)
    
    model.zero_constraints=ConstraintList() 
    for (i,j) in combinations(model.x,2):
        zero_add=(model.x[j] + model.x_dims[j] <= model.x[i]  + M_x * (1 - model.ors[i,j,0]))
        model.zero_constraints.add(expr=zero_add)
    
    model.one_constraints=ConstraintList()
    for (i,j) in combinations(model.x,2):
        one_add=(model.x[i] + model.x_dims[i] <= model.x[j] + M_x * (1 - model.ors[i,j,1]))
        model.one_constraints.add(expr=one_add)
        
    model.two_constraints=ConstraintList()
    for (i,j) in combinations(model.x,2):
        two_add=(model.y[j] + model.y_dims[j] <= model.y[i] + M_y * (1 - model.ors[i,j,2]))
        model.two_constraints.add(expr=two_add)
        
    model.three_constraints=ConstraintList()
    for (i,j) in combinations(model.x,2):
        three_add=(model.y[i] + model.y_dims[i] <= model.y[j] + M_y * (1 - model.ors[i,j,3]))
        model.three_constraints.add(expr=three_add)            
                
    model.x_biggest=Constraint(expr=model.x[biggest] <= (width//2))
    model.y_biggest=Constraint(expr=model.y[biggest]<= (model.height/2))
    
    #####################
    #Objective Definition
    #####################
    
    model.obj = Objective(expr= model.height, sense=minimize)
    model.write('RAP.lp') 
    
   
    #solver_list=['glpk','cbc','ipopt','couenne']
    solver_list=['glpk','cbc','ipopt']
    
    for i in range(len(solver_list)):
        if i==0:
            print(f'--- instance number {ins} ---\n')
        print(f'solver number {i+1} out of {len(solver_list)}: {solver_list[i]}')
        

        if solver_list[i]=='cbc':
            opt = SolverFactory('cbc',cbc_excutable='C:\\Cbc-2.10.5-x86_64-w64-mingw32\\bin')
            opt.options['sec'] = 300
    
        elif solver_list[i]=='glpk':
            opt = SolverFactory(solver_list[i])
            opt.options['tmlim'] = 300
            
        elif solver_list[i]=='couenne':
            
            opt = SolverFactory('couenne',excutable='C:\\Couenne-0.2.2-win32-msvc9\\bin\\couenne.exe')
            #time_limit (300) in couenne.opt
            
        elif solver_list[i]=='ipopt':
            opt = SolverFactory(solver_list[i])
            opt.options['max_cpu_time'] = 300
        
        tempo_initial = time.time()        
        
        results=opt.solve(model)
        tempo = time.time()-tempo_initial

        if tempo<300:
            with open('sol\ins{}_MIP_Rotation_solved_by_{}.txt'.format(ins,solver_list[i]), 'w') as myfile:
                myfile.write('time='+'{:.3f}'.format(tempo))
                myfile.close()
           
            print('feasible\n')
            for j in model.height:
                print('calculated height {} in {:.3f} seconds'.format(model.height[j].value,tempo))
            
            print('\n')


        else:
            with open('sol\ins{}_MIP_Rotation_solved_by_{}.txt'.format(ins,solver_list[i]), 'w') as myfile:
                myfile.write('infeasible\n')
                myfile.close()
            print('infeasible\n')
        
    ins+=1
    
    

--- instance number 1 ---

solver number 1 out of 3: glpk
feasible

calculated height 8.0 in 0.116 seconds


solver number 2 out of 3: cbc
feasible

calculated height 8.0 in 0.339 seconds


solver number 3 out of 3: ipopt
feasible

calculated height 8.0 in 0.703 seconds


--- instance number 2 ---

solver number 1 out of 3: glpk
feasible

calculated height 9.0 in 0.053 seconds


solver number 2 out of 3: cbc
feasible

calculated height 9.0 in 0.420 seconds


solver number 3 out of 3: ipopt
feasible

calculated height 9.0 in 0.063 seconds


--- instance number 3 ---

solver number 1 out of 3: glpk
feasible

calculated height 10.0 in 0.071 seconds


solver number 2 out of 3: cbc
feasible

calculated height 10.0 in 1.168 seconds


solver number 3 out of 3: ipopt
feasible

calculated height 10.0 in 0.057 seconds


--- instance number 4 ---

solver number 1 out of 3: glpk
feasible

calculated height 11.0 in 0.467 seconds


solver number 2 out of 3: cbc
feasible

calculated height 11.0 in 1.

--- instance number 30 ---

solver number 1 out of 3: glpk
infeasible

solver number 2 out of 3: cbc
    containing a solution
infeasible

solver number 3 out of 3: ipopt
feasible

calculated height 37.0 in 0.283 seconds


--- instance number 31 ---

solver number 1 out of 3: glpk
infeasible

solver number 2 out of 3: cbc
    containing a solution
infeasible

solver number 3 out of 3: ipopt
feasible

calculated height 38.0 in 0.143 seconds


--- instance number 32 ---

solver number 1 out of 3: glpk
infeasible

solver number 2 out of 3: cbc
    containing a solution
infeasible

solver number 3 out of 3: ipopt
feasible

calculated height 39.0 in 0.241 seconds


--- instance number 33 ---

solver number 1 out of 3: glpk
infeasible

solver number 2 out of 3: cbc
    containing a solution
infeasible

solver number 3 out of 3: ipopt
feasible

calculated height 40.0 in 0.146 seconds


--- instance number 34 ---

solver number 1 out of 3: glpk
infeasible

solver number 2 out of 3: cbc
    con